In [1]:
include("../ToolBox/ToolBox.jl")
using .ToolBox
import JLD
using Colors
import FileIO

In [2]:
#folder = "/Users/simon/Documents/DTU fag/Speciale/results/houston/river_BAB"
#folder = "/Users/simon/Documents/DTU fag/Speciale/results/houston/city_center_BAB"
#folder = "/Users/simon/Documents/DTU fag/Speciale/results/houston/resovoir_BAB"
#folder = "/Users/simon/Documents/DTU fag/Speciale/results/vejle_oktober_AAA"
#folder = "/Users/simon/Documents/DTU fag/Speciale/results/vejle_oktober/factory_ABA"
folder = "/Users/simon/Documents/DTU fag/Speciale/results/vejle_2020/factory"
#folder = "/Users/simon/Documents/DTU fag/Speciale/results/vejle_2020/factory_kolding"
#folder = "/Users/simon/Documents/DTU fag/Speciale/results/south_jutland_jan/vejle_1"
#folder = "/Users/simon/Documents/DTU fag/Speciale/results/south_jutland_jan/vejle_2"
#folder = "/Users/simon/Documents/DTU fag/Speciale/results/south_jutland_jan/ribe"

ascending = true
dem_nan = 40
# dem_nan=-25

40

In [3]:
if ascending 
   # Load
    co_gamma = JLD.load(joinpath(folder,"21VV_coher.jld"),"data")[end:-1:1,:]
    pre_gamma = JLD.load(joinpath(folder,"23VV_coher.jld"),"data")[end:-1:1,:]

    VV_paths = [joinpath(folder,string(i)*"VV_itens.jld") for i=1:3]
    VV = [JLD.load(path,"data")[end:-1:1,:] for path in VV_paths]
    VH_paths = [joinpath(folder,string(i)*"VH_itens.jld") for i=1:3]
    VH = [JLD.load(path,"data")[end:-1:1,:] for path in VH_paths];
    heights = JLD.load(joinpath(folder,"heights.jld"),"data")[end:-1:1,:].-dem_nan;
else
    # Load
    co_gamma = JLD.load(joinpath(folder,"21VV_coher.jld"),"data")[:,end:-1:1]
    pre_gamma = JLD.load(joinpath(folder,"23VV_coher.jld"),"data")[:,end:-1:1]

    VV_paths = [joinpath(folder,string(i)*"VV_itens.jld") for i=1:3]
    VV = [JLD.load(path,"data")[:,end:-1:1] for path in VV_paths]
    VH_paths = [joinpath(folder,string(i)*"VH_itens.jld") for i=1:3]
    VH = [JLD.load(path,"data")[:,end:-1:1] for path in VH_paths];
    heights = JLD.load(joinpath(folder,"heights.jld"),"data")[:,end:-1:1].-dem_nan;
    end;

In [4]:
filt = SlcUtil.temporal_filter([VV[1],VH[1]],5);

VV = SlcUtil.temporal_filter(VV,5);
VH = SlcUtil.temporal_filter(VH,5);

VV[1] = filt[1]
VH[1] = filt[2];

In [5]:
function db_scale_img(img , min , max)
    log_img = (10*log10.(img).-min)./(max-min)
    log_img[log_img.>1] .= 0.999
    log_img[log_img.<0] .= 0.001
    return log_img
end


function db_scale_img_diff(img1,img2 , min , max)
    log_img = 10*log10.(img1).-10*log10.(img2)
    log_img = (log_img.-min)./(max-min)
    log_img[log_img.>1] .= 0.999
    log_img[log_img.<0] .= 0.001
    return log_img
end

function scale_img(img , min , max)
    scale_img = (img.-min)./(max-min)
    scale_img[scale_img.>1] .= 0.999
    scale_img[scale_img.<0] .= 0.001
    return scale_img
end


function scale_img(img)
    min = minimum(reshape(img,:))
    max = maximum(reshape(img,:))
    return scale_img(img , min , max)
end


scale_img (generic function with 2 methods)

In [6]:
min_vv = -19
max_vv = 4

min_vh = -26
max_vh = -3

min_r = -10
max_r = 1;

min_h = -10
max_h = 100;

In [7]:
img = Colors.Gray.(scale_img(heights,min_h,max_h));
img_path = joinpath(folder,"dem.png")
FileIO.save(img_path,img);

In [8]:
img = Colors.Gray.(db_scale_img(VV[1],min_vv,max_vv));
img_path = joinpath(folder,"VV1.png")
FileIO.save(img_path,img);

img = Colors.Gray.(db_scale_img(VV[2],min_vv,max_vv));
img_path = joinpath(folder,"VV2.png")
FileIO.save(img_path,img);

img = Colors.Gray.(db_scale_img(VV[3],min_vv,max_vv));
img_path = joinpath(folder,"VV3.png")
FileIO.save(img_path,img);


img = Colors.Gray.(db_scale_img(VH[1],min_vh,max_vh));
img_path = joinpath(folder,"VH1.png")
FileIO.save(img_path,img);

img = Colors.Gray.(db_scale_img(VH[2],min_vh,max_vh));
img_path = joinpath(folder,"VH2.png")
FileIO.save(img_path,img);

img = Colors.Gray.(db_scale_img(VH[3],min_vh,max_vh));
img_path = joinpath(folder,"VH3.png")
FileIO.save(img_path,img);

In [9]:
img = Colors.Gray.(abs.(co_gamma));
img_path = joinpath(folder,"co_gamma.png")
FileIO.save(img_path,img);

img = Colors.Gray.(abs.(pre_gamma));
img_path = joinpath(folder,"pre_gamma.png")
FileIO.save(img_path,img);

In [10]:
img = Colors.RGB{Float32}.(abs.(pre_gamma),abs.(co_gamma),abs.(co_gamma));
img_path = joinpath(folder,"PreCoCo_coherence.png")
FileIO.save(img_path,img);

In [11]:
bands = [db_scale_img(elem,min_vv,max_vv) for elem in VV][end:-1:1]
img = Colors.RGB{Float32}.(bands...);
img_path = joinpath(folder,"Pre2Pre1Co_VV.png")
FileIO.save(img_path,img);

bands = [db_scale_img(elem,min_vh,max_vh) for elem in VH][end:-1:1]
img = Colors.RGB{Float32}.(bands...);
img_path = joinpath(folder,"Pre2Pre1Co_VH.png")
FileIO.save(img_path,img);

In [12]:
k = 1/1.4
pre = db_scale_img((VV[2] .+VV[3])./2,min_vv,max_vv) 
co = db_scale_img(VV[1],min_vv,max_vv) 
co = co.^k
pre = pre.^k
img = Colors.RGB{Float32}.(pre,co,co);
img_path = joinpath(folder,"haha_VV.png")
FileIO.save(img_path,img);

img = Colors.Gray.(db_scale_img_diff(co,pre,-3,3)); 
img_path = joinpath(folder,"Co_minus_pre_VV.png")
FileIO.save(img_path,img);

k = 1/1.4
pre = db_scale_img((VH[2] .+VH[3])./2,min_vh,max_vh) 
co = db_scale_img(VH[1],min_vh,max_vh)
co = co.^k
pre = pre.^k
img = Colors.RGB{Float32}.(pre,co,co);
img_path = joinpath(folder,"haha_VH.png")
FileIO.save(img_path,img);

img = Colors.Gray.(db_scale_img_diff(co,pre,-3,3)); 
img_path = joinpath(folder,"Co_minus_pre_VH.png")
FileIO.save(img_path,img);

In [13]:
bands = [[db_scale_img(VV[i],min_vv,max_vv),db_scale_img(VH[i],min_vh,max_vh),db_scale_img(VH[i]./VV[i],min_r,max_r)]
        for i in 1:3]

img = Colors.RGB{Float32}.(bands[1]...);
img_path = joinpath(folder,"Pol1.png")
FileIO.save(img_path,img);

img = Colors.RGB{Float32}.(bands[2]...);
img_path = joinpath(folder,"Pol2.png")
FileIO.save(img_path,img);

img = Colors.RGB{Float32}.(bands[3]...);
img_path = joinpath(folder,"Pol3.png")
FileIO.save(img_path,img);

In [14]:
delta_gamma = (abs.(co_gamma).-abs.(pre_gamma).+0.5)

delta_gamma[delta_gamma.<0] .= 0
delta_gamma[delta_gamma.> 0.5] .= 0.5
delta_gamma[abs.(pre_gamma) .<0.2] .= 0.5

delta_gamma = delta_gamma
k = 1/1.4
pre = db_scale_img((VV[2] .+VV[3])./2,min_vv,max_vv) 
co = db_scale_img(VV[1],min_vv,max_vv) 
co = co.^k
pre = pre.^k
img = Colors.RGB{Float32}.(pre,co,delta_gamma);
img_path = joinpath(folder,"combi_test_VV.png")
FileIO.save(img_path,img);


In [15]:
delta_gamma = (abs.(co_gamma).-abs.(pre_gamma).+0.5)

delta_gamma[delta_gamma.<0] .= 0
delta_gamma[delta_gamma.> 0.5] .= 0.5
delta_gamma[abs.(pre_gamma) .<0.2] .= 0.5

delta_gamma = delta_gamma
k = 1/1.4
pre = db_scale_img((VV[2] .+VV[3])./2,min_vv,max_vv) 
co = db_scale_img(VV[1],min_vv,max_vv) 
co = co.^k
pre = pre.^k
img = Colors.RGB{Float32}.(pre,co,delta_gamma);
img_path = joinpath(folder,"combi_test_VV.png")
FileIO.save(img_path,img);


In [16]:
pre = db_scale_img((VV[2] .+VV[3])./2,min_vv,max_vv) 
co = db_scale_img(VV[1],min_vv,max_vv) 
coVH = db_scale_img(VH[1],min_vh,max_vh) 
preVH = db_scale_img((VH[2] .+VH[3])./2,min_vh,max_vh) 

#wm1 = (pre .- co) .> 0.1
wm2 = co .< 0.3;
wm3 = (abs.(co_gamma).< 0.3)
wm4 = coVH .< 0.3;

wm_red =  wm2 .+ wm3 .+ wm4

wm_red = wm_red .> 2.5
red = copy(co)
red[wm_red] .= 1


#wm1 = coVH .< 0.2
#wm2 = co .< 0.2;
wm3 = pre .< 0.3;
wm4 = preVH .< 0.3;
wm5 = (abs.(pre_gamma).< 0.5)

wm = wm3 .+ 1.5 .* wm4 .+ 2 .* wm5

wm = wm .> 3
blue = copy(co)
blue[wm] .= 1

red[wm .& wm_red] .= 0


img = Colors.RGB{Float32}.(red,co,blue);
img_path = joinpath(folder,"water.png")

FileIO.save(img_path,img);